# 使用 Neural Network 來預測
---

In [19]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [20]:
# 銀行的檔案
trainFile = open("./Data/bank-full.csv","r") 
testFile = open("./Data/bank.csv","r")

# 吐掉第一行
trainFile.readline()
testFile.readline()


# 資料
trainOrgDataArray = []
testOrgDataArray = []
for line in trainFile:
    lineData = line.split(";")
    
    tempArray = []
    for data in lineData:
        tempArray.append(data.replace("\n",""))
    trainOrgDataArray.append(tempArray)

trainFile.close()
testFile.close()

## 資料愈處理的函數
---

In [21]:
def job(x):
    return {
        "\"admin.\"":         [1,0,0,0,0,0,0,0,0,0,0,0],
        "\"unknown\"":        [0,1,0,0,0,0,0,0,0,0,0,0],
        "\"unemployed\"":     [0,0,1,0,0,0,0,0,0,0,0,0],
        "\"management\"":     [0,0,0,1,0,0,0,0,0,0,0,0],
        "\"housemaid\"":      [0,0,0,0,1,0,0,0,0,0,0,0],
        "\"entrepreneur\"":   [0,0,0,0,0,1,0,0,0,0,0,0],
        "\"student\"":        [0,0,0,0,0,0,1,0,0,0,0,0],
        "\"blue-collar\"":    [0,0,0,0,0,0,0,1,0,0,0,0],
        "\"self-employed\"":  [0,0,0,0,0,0,0,0,1,0,0,0],
        "\"retired\"":        [0,0,0,0,0,0,0,0,0,1,0,0],
        "\"technician\"":     [0,0,0,0,0,0,0,0,0,0,1,0],
        "\"services\"":       [0,0,0,0,0,0,0,0,0,0,0,1]
    }.get(x,                  [1,0,0,0,0,0,0,0,0,0,0,0])

In [22]:
def marital(x):
    return {
        "\"married.\"":       [1,0,0],
        "\"divorced\"":       [0,1,0],
        "\"single\"":         [0,0,1]
    }.get(x,                  [1,0,0])

In [23]:
def education(x):
    return {
        "\"unknown\"":        [1,0,0,0],
        "\"secondary\"":      [0,1,0,0],
        "\"primary\"":        [0,0,1,0],
        "\"tertiary\"":       [0,0,0,1]
    }.get(x,                  [1,0,0,0])

In [24]:
def default(x):
    return {
        "\"yes\"":            [1,0],
        "\"no\"":             [0,1]
    }.get(x,                  [1,0])

In [25]:
def housing(x):
    return {
        "\"yes\"":            [1,0],
        "\"no\"":             [0,1]
    }.get(x,                  [1,0])

In [26]:
def loan(x):
    return {
        "\"yes\"":            [1,0],
        "\"no\"":             [0,1]
    }.get(x,                  [1,0])

In [27]:
def contact(x):
    return {
        "\"unknown\"":        [1,0,0],
        "\"telephone\"":      [0,1,0],
        "\"cellular\"":       [0,0,1]
    }.get(x,                  [1,0,0])

In [28]:
def month(x):
    return {
        "\"jan\"":            [1,0,0,0,0,0,0,0,0,0,0,0],
        "\"feb\"":            [0,1,0,0,0,0,0,0,0,0,0,0],
        "\"mar\"":            [0,0,1,0,0,0,0,0,0,0,0,0],
        "\"apr\"":            [0,0,0,1,0,0,0,0,0,0,0,0],
        "\"may\"":            [0,0,0,0,1,0,0,0,0,0,0,0],
        "\"jun\"":            [0,0,0,0,0,1,0,0,0,0,0,0],
        "\"jul\"":            [0,0,0,0,0,0,1,0,0,0,0,0],
        "\"aug\"":            [0,0,0,0,0,0,0,1,0,0,0,0],
        "\"sep\"":            [0,0,0,0,0,0,0,0,1,0,0,0],
        "\"oct\"":            [0,0,0,0,0,0,0,0,0,1,0,0],
        "\"nov\"":            [0,0,0,0,0,0,0,0,0,0,1,0],
        "\"dec\"":            [0,0,0,0,0,0,0,0,0,0,0,1]
    }.get(x,                  [1,0,0,0,0,0,0,0,0,0,0,0])

In [29]:
def poutcome(x):
    return {
        "\"unknown\"":        [1,0,0,0],
        "\"other\"":          [0,1,0,0],
        "\"failure\"":        [0,0,1,0],
        "\"success\"":        [0,0,0,1]
    }.get(x,                  [1,0,0,0])

In [30]:
def termDeposit(x):
    return {
        "\"yes\"":            1,
        "\"no\"":             0
    }.get(x,                  1)

## 轉乘資料形式
---

In [40]:
# 將某一筆資料，全部轉乘 Training 的資料形式
def TransformDataToTraining(DataList):
    OutputArray = []
    
    OutputArray.extend([int(DataList[0])])
    OutputArray.extend(job(DataList[1]))
    OutputArray.extend(marital(DataList[2]))
    OutputArray.extend(education(DataList[3]))
    OutputArray.extend(default(DataList[4]))
    OutputArray.extend([int(DataList[5])])
    OutputArray.extend(housing(DataList[6]))
    OutputArray.extend(loan(DataList[7]))
    OutputArray.extend(contact(DataList[8]))
    OutputArray.extend([int(DataList[9])])
    OutputArray.extend(month(DataList[10]))
    OutputArray.extend([int(DataList[11])])
    OutputArray.extend([int(DataList[12])])
    OutputArray.extend([int(DataList[13])])
    OutputArray.extend([int(DataList[14])])
    OutputArray.extend(poutcome(DataList[15]))
    
    Ans = termDeposit(DataList[16])
    return OutputArray, Ans

In [44]:
# 要找出最小跟最大的陣列
def FindMinAndMax(DataList):
    OutputArray = []
    
    # 先把每一個的值拿進來
    for i in range(0, len(DataList[0])):
        # 產生暫存的 Array
        tempArray = []
        
        for dataNum in range(0, len(DataList)):
            tempArray.append(DataList[dataNum][i])
        OutputArray.append([min(tempArray), max(tempArray)])
    return OutputArray

In [45]:
# Feature 資料
FeatureDataList = []

# Ans
AnsDataList = []

for i in range(0, len(trainOrgDataArray)):
    FeatureData, AnsData = TransformDataToTraining(trainOrgDataArray[i])
    FeatureDataList.append(FeatureData)
    AnsDataList.append(AnsData)
    
# 找出最大最小值，為了要 Normalize
MinMaxArrayList = FindMinAndMax(FeatureDataList)
for i in range(0, len(trainOrgDataArray)):
    for j in range(0, len(trainOrgDataArray[0])):
        